# TX00DQ05-3001 Exercises 3

In [1]:
import numpy as np
import numpy.linalg as LA
import random
from IPython.core.debugger import Pdb
ipdb = Pdb()
random.seed(a=None, version=2)


## Exercise 1: Sample behaviour of an MDP

Let's take (again) a look at Sutton & Barto example 4.1 gridworld. On each iteration start at every (non-terminating) state and sample actions in succeeding states by selecting them from uniform distribution (each action - up, down, left, right - is equally probable). Run the episode until terminal state is encountered. Collect statistics to calculate average number of steps needed before completion for each start state. Should this number match with something you have seen earlier in the exercises?

In [2]:
# YOUR CODE

        
        
rows_count = 4
columns_count = 4
V = np.zeros((rows_count, columns_count))
reward = -1 
directions = ['up', 'right', 'down', 'left'] #probably not needed
maxiters = 10000
eps = 0.0000001
k = 0 # "memory counter" of iterations inside the for loop, note that for loop i-variable is regular loop variable

rows = 4
cols = 4

stepsMatrix = np.zeros((rows_count, columns_count))



In [3]:




def isTerminal(r,c):      #helper function to check if terminal state or regular state
    global rows_count, columns_count
    if r == 0 and c == 0: #im a bit too lazy to check otherwise the iteration boundaries        
        return True       #so that this helper function is a quick way to exclude computations
    if r == rows_count-1 and c == columns_count-1:
        return True
    return False

def getValue(row, col):    #helper func, get state value
    global V
    if row == -1: row =0   #if you bump into wall, you bounce back
    elif row == 4: row = 3
    if col == -1: col = 0
    elif col == 4: col =3
        
    return V[row,col]

def getState(row,col):
    if row == -1: row =0   #helper func for the exercise:1
    elif row == 4: row = 3
    if col == -1: col = 0
    elif col == 4: col =3
    return row, col


def makeEpisode(r,c):  #helper func for the exercise:1
## return the count of steps ??
#by definition, you should always start from non-terminal state, so
#by minimum, you need at least one action to get to terminal state
    stateWasTerm = False
    stepsTaken = 0
    curR = r
    curC = c
    while not stateWasTerm:

        act = random.randint(0,3)
        if act == 0: ##up
            curR-=1
        elif act == 1: ##right
            curC+=1
        elif act == 2: ## down
            curR+=1
        else:##left
            curC-=1
        stepsTaken +=1
        curR,curC = getState(curR,curC)
        stateWasTerm = isTerminal(curR,curC)
    return stepsTaken
        

In [4]:
for n in range(maxiters):
    
    for r in range(rows_count): ##for all states
        for c in range(columns_count):
            if isTerminal(r, c): ##if was terminal states => nothing
                continue
            else:               ##else  => do useful things
                episodeSteps = makeEpisode(r,c)
                temp = stepsMatrix[r,c]
                temp += episodeSteps
                stepsMatrix[r,c] = temp
               ##temp = stepsDict[(r,c)]
                ##temp += episodeSteps
                ##stepsDict[(r,c)] = temp

#update the sumStepsTaken for each state, into the averages over episodes                
for r in range(rows):
    for c in range(cols):
        stepsMatrix[r,c] /= maxiters

print(stepsMatrix)



[[ 0.     14.3512 19.8079 22.3901]
 [13.9732 18.0939 19.9505 19.7331]
 [19.7774 20.1481 17.9906 13.662 ]
 [22.186  20.1546 13.8741  0.    ]]


## answer for monte carlo thingy

* when computing for each state, ( (sum of episodicSteps) / episodes)
* those values inside their proper places in the numpy matrix looks like they are close to the absolute value of (state-value function)
* in regular gridworld the values were the same except they were negative, because in gridworld there was always reward = -1
* NOTE!, maxiters was only 10k initially, becvause I kept it low, so that I could better find out what was happening later on in this notebook in the ex2 monte carlo, where I used episodeCount variable instaed to control the iterations. I kept the maxiters lower, so the notebook would run faster, and I kept episodeCount higher so I got more reliable monte carlo's, I think...

## Exercise 2: Monte Carlo state value function estimation. 

Calculate state-value function V for the gridworld of Sutton & Barto example 4.1 using first-visit or every-visit Monte Carlo policy evaluation (see for example page 92 of Sutton & Barto). Policy to be evaluated is the same as before; each action (up, down, left, right) is equally probable.  Action that would result in leaving the grid (for example moving up in top row) will leave state unchanged (but action has been taken). Gamma (discount factor) is assumed to be = 1, ie. no discounting.

Try out both exploring starts (see Sutton & Barto, p. 96) and fixed start points. Any difference?

Take a look at the value function you get when you run the algorithm multiple times (with fixed # of iterations). Any observations?

## NOTE! I will only do every-visit monte carlo (fixed start_loc) because...
*  I didn't undertstand how to do first visit monte carlo. The checking for the first-visit in the pseudocode didn't seem to ring any bells in my head...

In [5]:
"""some initializations, for datastructures and variables"""
V = np.zeros((rows_count, columns_count))
episodeCount = 100000*2
reward = -1
y = 1.0 #the gamma discount rate


#use dictionary where key is stateTuple, 
#and value is stateReturnsList
#after algorithm for monte carlo policy eval is done, 
#we can update the dict into good format for printing
#and use numpy matrix
returnsDict={} 
for r in range(4):
    for c in range(4):
        if not isTerminal(r,c):
            returnsDict[(r,c)]=[]

        
print(returnsDict)

    

{(0, 1): [], (0, 2): [], (0, 3): [], (1, 0): [], (1, 1): [], (1, 2): [], (1, 3): [], (2, 0): [], (2, 1): [], (2, 2): [], (2, 3): [], (3, 0): [], (3, 1): [], (3, 2): []}


In [6]:
""" montecarlo episode generation
returns the episodelist"""
def MCEpisode(r,c):
    global reward
    stateWasTerm = False
    stepsTaken = 0
    curR = r 
    curC = c
    episodeList=[]
    #visitedStates=[]
    
    while not stateWasTerm:

        act = random.randint(0,3)
        if act == 0: ##up
            r-=1
            act="U"
        elif act == 1: ##right
            c+=1
            act="R"
        elif act == 2: ## down
            r+=1
            act="D"
        else:##left
            c-=1
            act="L"
        stepsTaken +=1
        
        r,c = getState(r,c)
        stateWasTerm = isTerminal(r,c)
        episodeList.append( ((curR,curC), act, reward) )
        #visitedStates.append( (curR,curC)  )
        if not stateWasTerm:
            #episodeList.append( ((curR,curC), act, reward) )
            curR = r
            curC = c
        
        
    return episodeList

In [7]:
def getRandomStartState():
    illegalState = True

    
    while illegalState:
        r = random.randint(0,3)
        c = random.randint(0,3)
        
        if (r==0 and c==0) or (r==3 and c==3):
            illegalState = True
        else:
            illegalState= False
    return r,c
    

In [8]:
"""every-visit Monte Carlo with fixed starting state in the s(1,1) state
or alternatively start from random state (sR,sC)"""
kakka=0 #for debug breakpoints only!

for n in range(1, episodeCount+1):
    #ipdb.set_trace() # debugging starts here
    epList = MCEpisode(1,1)
    G = 0
    for t in reversed( range( len(epList) )):
        
        G = y*G + reward #NOTE! reward is always same -1
        S_t = epList[t][0] #get the state only, from tuple
        willAppend = True
        if(willAppend):
            returnsDict[S_t].append(G)
        
    
kakka = 3 #for debug breakpoints only!

#print(returnsDict)

    
    
    
    
    
    
    
    
    
    
    
    
    

In [9]:
"""I think the average was for every-visit MC as follows:
for each state
    get the list of returns and sum the elements together,
    divide that sum, by the amount of elements that were in the list

This will not work, IF BY VERY UNLUCKY EVENT, we will have missesd entirely some state, 
across all the episodes... because...
because then... we would get DivisionByZeroError..."""

for r in range(4):
    for c in range(4):
        if not isTerminal(r,c):
        #divisor = len(returnsDict[(r,c)])
        #if divisor == 0:
        #    divisor+=1
            V[r,c] = sum(returnsDict[(r,c)]) / ( len(returnsDict[(r,c)]) ) # adding +1 to prevent divisionByZero, not all states 
                                                     # were visited apparently...???
"""there used to be a divisionByZero error at this stage, but 
most likely it was caused by iterating over the terminalStates somehow,

theoretically it would be possible to have divisionbyZero error also in non-terminal state
but that would have required that across all episodes, there was some state in 14 non-terminal-states that
was never visited...

That could have happened if episodeCount is low, and then you never visit some particular state,
which leaves that state's own list empty, which causes the length of that list to be zero =>
divisionByZeroError

it will be unlikely to happen if episodesCount is high for Monte-Carlo"""
print("fixed start_loc Monte Carlo, every-visit\n")            
print(V)
    

fixed start_loc Monte Carlo, every-visit

[[  0.         -13.94935302 -19.97969824 -22.06874196]
 [-14.12209103 -18.04649906 -19.96745157 -19.98042336]
 [-20.15933997 -20.07762604 -18.09017819 -14.06051972]
 [-22.05534641 -20.13186602 -14.20269952   0.        ]]


## results for  every-visit Monte-Carlo (fixed start_loc)
* results look similar to the state value function from the chapter 4 exercises with MDP style algorithms for state value function for randomwalk policy. Not, quite equal, but good enough I hope?
* it might be possible that the value function is little bit skewed towards favouring the upper left corner better
* this is because starting state was fixed to be (1,1) because Sutton&Barto did not specify in the pseudocode if the starting state  should be randomized or not, or if you should vary starting state somehow...

* so that if upper left corner has bigger state values (closer towards zero) then that might be cause of the favoritism of starting loc == (1,1)

* because the grid was 4x4, it will be impossible to choose centerpoint that is equal for all sides, because it is missing the "center element" which would equally divide the array in length and width

## how to do every-visit Monte-Carlo (exploring starts)
* that's the million dollar question isn't it
* I will attempt to do it now
* by definition it was so that in exploring starts...
* every state-action pair has to be visited very often (equally often???)


#### Monte Carlo ES (Exploring Starts), for estimating policy Pi ~~ Pi* (p. 99 )

* in the pseudocode, it was mentioned that it will estimate policy into the optimal policy eventually.'
* it also used Q(s,a) instead of V(s)
* but, the crucial difference seemed to be that in Exploring starts you select starting state randomly, and starting action randomly, with non-zero probabilities (I wonder, if it means you can select the starting state and starting action with the original equiprobable thingy, such as 1/14 chance  of non-terminating random state, and 1/4 action)

* I think we could still attempt to do the exploring starts case, but simply use it for the V(s) and then just randomize starting state.

* the expectations is that  for V(s) will be that with the same amount of episodes run, the Exploring starts V(s) should be in equilibrium just like it was in chapter 4 MDP exercises V(s). This is in stark difference with fixed start Monte Carlo which was slanted to upper left corner

In [10]:
"""resetting the variables and datastructuires for new round"""
V = np.zeros((rows_count, columns_count))
returnsDict={} 
for r in range(4):
    for c in range(4):
        if not isTerminal(r,c):
            returnsDict[(r,c)]=[]

print(V)
print(returnsDict)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
{(0, 1): [], (0, 2): [], (0, 3): [], (1, 0): [], (1, 1): [], (1, 2): [], (1, 3): [], (2, 0): [], (2, 1): [], (2, 2): [], (2, 3): [], (3, 0): [], (3, 1): [], (3, 2): []}


In [11]:
"""every-visit Monte Carlo with start from random state (sR,sC)"""
kakka=0 #for debug breakpoints only!

for n in range(1, episodeCount+1):
    #ipdb.set_trace() # debugging starts here
    sR, sC = getRandomStartState()
    epList = MCEpisode(sR,sC)
    G = 0
    for t in reversed( range( len(epList) )):
        
        G = y*G + reward #NOTE! reward is always same -1
        S_t = epList[t][0] #get the state only, from tuple
        willAppend = True
        if(willAppend):
            returnsDict[S_t].append(G)
        
    
kakka = 3 #for debug breakpoints only!

#print(returnsDict)




In [12]:
for r in range(4):
    for c in range(4):
        if not isTerminal(r,c):
        #divisor = len(returnsDict[(r,c)])
        #if divisor == 0:
        #    divisor+=1
            V[r,c] = sum(returnsDict[(r,c)]) / ( len(returnsDict[(r,c)]) ) # adding +1 to prevent divisionByZero, not all states 
                                                     # were visited apparently...???
"""there used to be a divisionByZero error at this stage, but 
most likely it was caused by iterating over the terminalStates somehow,

theoretically it would be possible to have divisionbyZero error also in non-terminal state
but that would have required that across all episodes, there was some state in 14 non-terminal-states that
was never visited...

That could have happened if episodesCouint is low, and then you never visit some particular state,
which leaves that state's own list empty, which causes the length of that list to be zero =>
divisionByZeroError

it will be unlikely to happen if episodesCount is high for Monte-Carlo"""
print("exploring starts Monte Carlo, every-visit\n")           
print(V)

exploring starts Monte Carlo, every-visit

[[  0.         -13.96630385 -19.9212714  -21.90385924]
 [-13.91160224 -17.99611573 -20.03500858 -20.0187521 ]
 [-19.9440678  -19.97834431 -18.05784158 -13.99097402]
 [-21.94451288 -19.96657264 -14.05662013   0.        ]]


## results for every-visit Monte-Carlo (exploring starts)

* I decided to turn up the episodecount to 500k
* this had the effect of actually removing the skewed value function for fixed start_loc V(s) MC
* also the results for randomized start loc V(s) MC were quite similar.
* no big differences between those V(s) in my opinion

* 500k episodecouint is very laggy though, for python to execute

## Exercise 3*: Monte Carlo action value function estimation

Use the same idea as in exercise 2 to estimate q function.

*) - not mandatory

## Exercise 4*: Monte Carlo control

Compute the optimal policy for the 4x4 gridworld example. Start with random policy. Consider the epsilon adjustment schedule - can it in practise be 1/k, or is something more conservative better? Can you think of any other tricks to manage the noisiness of MC?

*) - not mandatory